In [2]:
!pwd

/content


In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [4]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-dh5omjzg
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-dh5omjzg
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=87a0c34058ed34976ddb797d322796d187b1a8ac449778913e2e6f87d8c29d85
  Stored in directory: /tmp/pip-ephem-wheel-cache-bq6jaumv/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [39]:
%%cuda --name prg1.cu

#include<stdio.h>
#include "device_launch_parameters.h"
#include "cuda_runtime.h"

//device functions are signified by __global__ and they are always void
__global__ void add(int *a,int *b,int *c)
{
    //in this question,each thread performs addition operation of each coefficient in the vector.
    int i=(blockIdx.x*blockDim.x)+threadIdx.x;  //x because we only have blocks and threads in the x direction in this case.
    c[i]=a[i]+b[i];
}


int main(void)
{
    int n,i;
    printf("enter size of n");
    scanf("%d",&n);
    //define size required for gpu variables
    int size=n*sizeof(int);
    int a[n],b[n],c[n];
    int *d_a,*d_b,*d_c; //declare device variables
    //allocate gpu memory to devce variables.
    cudaMalloc((void**)&d_a,size);
    cudaMalloc((void**)&d_b,size);
    cudaMalloc((void**)&d_c,size);
    printf("enter array 1:");
    for(i=0;i<n;i++)
    scanf("%d",&a[i]);
    printf("enter array 2:");
    for(i=0;i<n;i++)
    scanf("%d",&b[i]);
    //allocate memeory for both arrays in the gpu
    //copy the values in a and b to device variables.
    cudaMemcpy(d_a,&a,size,cudaMemcpyHostToDevice);
    cudaMemcpy(d_b,&b,size,cudaMemcpyHostToDevice);
    //define how many blocks you ant in your grid==grid dimension (it is a structure specifying num of blocks in each direction)
    dim3 dimGrid(1,1,1);
    //define number of threads per block==block dimension
    dim3 dimBlock(n,1,1);
    //launch the kernel function
    add<<<dimGrid,dimBlock>>>(d_a,d_b,d_c);
    //after kernel launch, copy the resultant values back to cpu memory.
    cudaMemcpy(&c,d_c,size,cudaMemcpyDeviceToHost);
    printf("resultant vector:");
    for(i=0;i<n;i++)
    printf("%d ",c[i]);
    //Free the memeory allocated to device variables.
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    return 0;
}


'File written in /content/src/prg1.cu'

In [40]:
!nvcc /content/src/prg1.cu -o /content/src/prg1

In [41]:
!/content/src/prg1

enter size of n3
enter array 1:1 1 1
enter array 2:2 2 2
resultant vector:3 3 3 